In [ ]:
!nvidia-smi

Sun May 28 21:21:21 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Load the Drive helper and mount:
from google.colab import drive

# This will prompt for authorization:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -U segmentation-models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 5.0 MB/s eta 0:00:00


In [ ]:
import segmentation_models as sm

AttributeError: ignored

 changed keras.utils.generic_utils.get_custom_objects().update(custom_objects) to keras.utils.get_custom_objects().update(custom_objects) in .../lib/python3.6/site-packages/efficientnet/__init__.py and it solved the issue.

In [ ]:
import keras
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

2.12.0
2.12.0


In [ ]:
#from keras.utils import normalize
import os
import glob
import cv2
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from keras.metrics import MeanIoU
from keras import backend as K
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint, TensorBoard, ReduceLROnPlateau, EarlyStopping
import matplotlib.colors
import pandas as pd
from keras.models import load_model

In [ ]:
def divide_2d_list(input_list, divisor):
    result = []
    for row in input_list:
        result.append([value / divisor for value in row])
    return result

In [ ]:
def load_images(image_paths):
    images = []
    for path in image_paths:
        image = Image.open(path)
        if image.mode != 'RGB':
            image = image.convert('RGB')
        images.append(np.array(image))
    return np.array(images)

In [ ]:
def load_npy_files(file_list):
    arrays = [np.load(file) for file in file_list]
    return np.array(arrays)

In [ ]:
def load_data(folder):
    image_names = glob.glob(os.path.join(folder, "*.png"))
    image_names.sort()
    image_dataset = load_images(image_names)
    #image_dataset = np.expand_dims(image_dataset, axis = 3)
    mask_names = glob.glob(os.path.join(folder, "*.npy"))
    mask_names.sort()
    mask_dataset = load_npy_files(mask_names)
    mask_dataset_encoded = np.expand_dims(mask_dataset, axis = 3)
    #Normalize images
    #image_dataset = image_dataset /255.  #Can also normalize or scale using MinMax scaler
    train_masks_cat = to_categorical(mask_dataset_encoded, num_classes=n_classes)
    gt_data = train_masks_cat.reshape((mask_dataset_encoded.shape[0], mask_dataset_encoded.shape[1], mask_dataset_encoded.shape[2], n_classes))

    return image_dataset, gt_data, mask_dataset_encoded

In [ ]:
def get_metrics(model, images, groud_truth, n_classes=5, class_names=['background', 'lesion', 'marker', 'ruler', 'skin']):
    preds = model.predict(images)
    preds = np.argmax(preds, axis=3)

    ## Get metrics (overall)
    IOU_keras = MeanIoU(num_classes=n_classes)
    IOU_keras.update_state(groud_truth[:,:,:,0], preds)
    mean_IOU = IOU_keras.result().numpy()
    values = np.array(IOU_keras.get_weights()).reshape(n_classes, n_classes)

    ious = []
    for i in range(n_classes):
        numerator = values[i,i]
        denominator = np.sum(values, axis=1)[i] + (np.sum(values, axis=0)[i] - values[i,i])
        iou = numerator/denominator
        ious.append(iou)
        print("Train IoU for", class_names[i], iou)

    return mean_IOU, ious, preds

In [ ]:
def get_metrics_per_tone(model, path):
    imgs, _, gt = load_data(path)
    imgs = preprocess_input(imgs)
    mean, per_class, _ = get_metrics(model, imgs, gt)
    return mean, per_class

In [ ]:
def save_preds(preds, dir, split, cmap):
    for i, pred in enumerate(preds):
        plt.imshow(pred, cmap=cmap, vmin=0, vmax=4)
        if split == 'val':
            dest = os.path.join(dir, "validation_visualizations")
            print("save at", dest)
            os.makedirs(dest, exist_ok=True)
            plt.savefig(os.path.join(dest, f"{i}.pdf"))
        elif split == 'test':
            dest = os.path.join(dir, "visualizations")
            os.makedirs(dest, exist_ok=True)
            print("save at", dest)
            plt.savefig(os.path.join(dest, f"{i}.pdf"))
        plt.show()

In [ ]:
def plot_images(images, max_cols=5, axis=True, cmap=None, init='/content/3J6A0789.JPG',
                save=False, path=None):
  # Calculate the number of rows and columns needed
  num_images = len(images)
  num_cols = min(num_images, max_cols)
  num_rows = (num_images - 1) // num_cols + 1

  # Create a figure with a grid of subplots
  fig, axs = plt.subplots(num_rows, num_cols, figsize=(20,10))

  # Plot each image in a subplot
  for i, ax in enumerate(axs.flat):
    if i < num_images:
      if init and i == 0:
        size = np.array(images[0])[0].shape[-1]
        img = read_image(init, size=(size, size), ch_first=False)
        ax.imshow(img)
      else:
        if cmap == 'gray':
          try:
            ax.imshow(images[i], cmap='gray')
          except:
            ax.imshow(images[i].cpu(), cmap='gray')
        elif cmap and (i%2):
          ax.imshow(images[i], cmap=cmap, vmin=0, vmax=4)
        else:
          ax.imshow(images[i])
      if not axis:
        ax.axis('off')

  if save and path:
      print("save at", os.path.join(path, "testset_visualizations.pdf"))
      plt.savefig(os.path.join(path, "testset_visualizations.pdf"))
  plt.show()

In [ ]:
exp_table_path = os.path.join("/content/drive/MyDrive/Winter 2023/DDPM/",
                              "experiments_unet_rebuttalv2.csv")

try:
    experiment_table = pd.read_csv(exp_table_path)

except:
    headers = {"Method":[], "DDI Fraction (%)":[], "Fold":[], "Zoom":[],
               "Image Size":[], "Diffusion Timesteps":[], "Blocks":[],
               "Embedding Size":[], "MLP Models":[],

               "Test Overall Mean IOU":[],
               "Test Light Mean IOU":[], "Test Medium Mean IOU":[],
               "Test Dark Mean IOU":[],

               "Test Light Lesion IOU":[], "Test Medium Lesion IOU":[],
               "Test Dark Lesion IOU":[],

               "Test Light Marker IOU":[], "Test Medium Marker IOU":[],
               "Test Dark Marker IOU":[],

               "Test Light Ruler IOU":[], "Test Medium Ruler IOU":[],
               "Test Dark Ruler IOU":[],

               "Test Light Skin IOU":[], "Test Medium Skin IOU":[],
               "Test Dark Skin IOU":[],

               "Test Light Background IOU":[], "Test Medium Background IOU":[],
               "Test Dark Background IOU":[]}

    table = pd.DataFrame.from_dict(headers)
    table.to_csv(exp_table_path, index=False)
    experiment_table = pd.read_csv(exp_table_path)

#Resizing images, if needed
SIZE_X = 256
SIZE_Y = 256
n_classes = 5 # Number of classes for segmentation

class_names = ['background', 'lesion', 'marker', 'ruler', 'skin']

palette = [[255, 255, 255],
           [255, 74,  70],
           [238, 130, 238],
           [0,   111, 166],
           [60,  179, 113]]

palette = divide_2d_list(palette, 255)
my_cmap = matplotlib.colors.ListedColormap(palette, name='viz_map')

In [ ]:
zoom = "Resized"
for BACKBONE in ['densenet121', 'vgg16', 'resnet50', 'efficientnetb0', 'mobilenetv2']:
    for perc in [20, 15, 10, 5]:
        for fold in [3]:
            # root_folder = f"/content/drive/MyDrive/Winter 2023/DDI/Segmentation Labels/{zoom}/Fold {fold}/{perc}%/"
            # train_folder = os.path.join(root_folder, "train")
            # val_folder = os.path.join(root_folder, "val")
            # test_folder = os.path.join(root_folder, "test")

            activation='softmax'

            LR = 0.0001
            optim = tf.keras.optimizers.Adam(LR)

            # Segmentation models losses can be combined together by '+' and scaled by integer or float factor
            focal_factor = 1
            dice_factor = 1

            dice_loss = sm.losses.DiceLoss()
            focal_loss = sm.losses.CategoricalFocalLoss()
            total_loss = (dice_factor * dice_loss) + (focal_factor * focal_loss)

            #total_loss = sm.losses.JaccardLoss()

            # actulally total_loss can be imported directly from library, above example just show you how to manipulate with losses
            # total_loss = sm.losses.binary_focal_dice_loss # or sm.losses.categorical_focal_dice_loss

            metrics = [sm.metrics.IOUScore(), sm.metrics.FScore()]
            #metrics = sm.metrics.IOUScore()

            ###Model
            preprocess_input = sm.get_preprocessing(BACKBONE)

            # preprocess input
            # raw_train_images, train_gt, train_viz = load_data(train_folder)
            # train_images = preprocess_input(raw_train_images)

            # raw_val_images, val_gt, val_viz = load_data(val_folder)
            # val_images = preprocess_input(raw_val_images)

            # raw_test_images, test_gt, test_viz = load_data(test_folder)
            # test_images = preprocess_input(raw_test_images)

            # define model
            model = sm.Unet(BACKBONE, encoder_weights='imagenet', classes=n_classes,
                            activation=activation, input_shape=(256,256,3))

            # compile keras model with defined optimozer, loss and metrics
            model.compile(optim, total_loss, metrics=metrics)

            #model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=metrics)

            model.summary()

            save_path = f"/content/drive/MyDrive/Winter 2023/DDPM/pixel_classifiers/unet/{BACKBONE}/{zoom}/Fold {fold}/{perc}%/"
            best_model_path = os.path.join(save_path, "final_model.hdf5")

            earlystop = EarlyStopping(monitor = 'val_iou_score',
                                    mode = "max",
                                    min_delta = 0,
                                    patience = 100,
                                    verbose = 1,
                                    restore_best_weights = True)

            reduce_lr = ReduceLROnPlateau(monitor='val_iou_score',
                                        mode = "max",
                                        factor = 0.1,
                                        patience = 50,
                                        min_lr = 0.00001,
                                        verbose = 1)

            tensor_board = tbCallBack = TensorBoard(log_dir = os.path.join(save_path, 'logs/'),
                                                    histogram_freq = 0,
                                                    write_graph = True,
                                                    write_images = False)


            # best_model = ModelCheckpoint(save_path,
            #                              monitor = 'val_loss',
            #                              mode = 'auto',
            #                              verbose = 1,
            #                              save_freq = 25,
            #                              save_best_only = True)

            callback_list = [reduce_lr, earlystop, tensor_board]

            # history = model.fit(train_images, train_gt,
            #                     batch_size=64,
            #                     epochs=1000,
            #                     verbose=0,
            #                     validation_data=(val_images, val_gt),
            #                     callbacks = callback_list)

            #model.save(best_model_path, overwrite=True)
            from keras.models import load_model
            model = load_model(best_model_path, compile=False)

            #plot the training and validation accuracy and loss at each epoch
            # loss = history.history['loss']
            # val_loss = history.history['val_loss']
            # epochs = range(1, len(loss) + 1)
            # plt.plot(epochs, loss, label='Training loss')
            # plt.plot(epochs, val_loss, label='Validation loss')
            # title = 'Training and validation loss'
            # plt.title(title)
            # plt.xlabel('Epochs')
            # plt.ylabel('Loss')
            # plt.legend()
            # plt.savefig(os.path.join(save_path, title + '.pdf'))
            # plt.show()

            # acc = history.history['iou_score']
            # val_acc = history.history['val_iou_score']

            # plt.plot(epochs, acc, label='Training IOU')
            # plt.plot(epochs, val_acc, label='Validation IOU')
            # title = 'Training and validation IOU'
            # plt.title(title)
            # plt.xlabel('Epochs')
            # plt.ylabel('IOU')
            # plt.legend()
            # plt.savefig(os.path.join(save_path, title + '.pdf'))
            # plt.show()

            # Overall
            # train_mean, train_per_class, train_preds = get_metrics(model, train_images, train_viz)
            # val_mean, val_per_class, val_preds = get_metrics(model, val_images, val_viz)
            # test_mean, test_per_class, test_preds = get_metrics(model, test_images, test_viz)

            base = f"/content/drive/MyDrive/Winter 2023/DDI/Segmentation Labels/{zoom}/Rebuttal/Fold {fold}/"

            # # val light
            # val_light_mean, val_light_per_class = get_metrics_per_tone(model, os.path.join(base, "val_light"))

            # # val med
            # val_med_mean, val_med_per_class = get_metrics_per_tone(model, os.path.join(base, "val_med"))

            # # val dark
            # val_dark_mean, val_dark_per_class = get_metrics_per_tone(model, os.path.join(base, "val_dark"))

            # test light
            test_light_mean, test_light_per_class = get_metrics_per_tone(model, os.path.join(base, "test_light"))

            # test med
            test_med_mean, test_med_per_class = get_metrics_per_tone(model, os.path.join(base, "test_med"))

            # test dark
            test_dark_mean, test_dark_per_class = get_metrics_per_tone(model, os.path.join(base, "test_dark"))

            # save viz
            # save_preds(test_preds, save_path, "test", my_cmap)
            # save_preds(val_preds, save_path, "val", my_cmap)

            # viz = []
            # for test_img, test_pred in zip(raw_test_images, test_preds):
            #     viz.append(test_img)
            #     viz.append(test_pred)
            # plot_images(viz, max_cols=8, cmap=my_cmap, init=None, save=True, path=save_path)

            test_mean = (test_light_mean + test_med_mean + test_dark_mean) / 3

            experiment_table.loc[len(experiment_table)] = [f"{BACKBONE}-Unet", perc,
                                                    fold, zoom, "[256, 256]",
                                                    "N/A", "N/A", "N/A", "N/A",


                                                    test_mean, test_light_mean,
                                                    test_med_mean, test_dark_mean,

                                                    test_light_per_class[1],
                                                    test_med_per_class[1],
                                                    test_dark_per_class[1],

                                                    test_light_per_class[2],
                                                    test_med_per_class[2],
                                                    test_dark_per_class[2],

                                                    test_light_per_class[3],
                                                    test_med_per_class[3],
                                                    test_dark_per_class[3],

                                                    test_light_per_class[4],
                                                    test_med_per_class[4],
                                                    test_dark_per_class[4],

                                                    test_light_per_class[0],
                                                    test_med_per_class[0],
                                                    test_dark_per_class[0]
                                                    ]

            experiment_table.to_csv(exp_table_path, index=False)

Streaming output truncated to the last 5000 lines.
 bn_data (BatchNormalization)   (None, 256, 256, 3)  9           ['data[0][0]']                   
                                                                                                  
 zero_padding2d_26 (ZeroPadding  (None, 262, 262, 3)  0          ['bn_data[0][0]']                
 2D)                                                                                              
                                                                                                  
 conv0 (Conv2D)                 (None, 128, 128, 64  9408        ['zero_padding2d_26[0][0]']      
                                )                                                                 
                                                                                                  
 bn0 (BatchNormalization)       (None, 128, 128, 64  256         ['conv0[0][0]']                  
                                )                         

/usr/local/lib/python3.10/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


9406464/9406464 [==============================] - 0s 0us/step
Model: "model_20"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_13 (InputLayer)          [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1_pad (ZeroPadding2D)      (None, 257, 257, 3)  0           ['input_13[0][0]']               
                                                                                                  
 Conv1 (Conv2D)                 (None, 128, 128, 32  864         ['Conv1_pad[0][0]']              
                                )                                                                 
                            

/usr/local/lib/python3.10/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "model_21"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_14 (InputLayer)          [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1_pad (ZeroPadding2D)      (None, 257, 257, 3)  0           ['input_14[0][0]']               
                                                                                                  
 Conv1 (Conv2D)                 (None, 128, 128, 32  864         ['Conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

/usr/local/lib/python3.10/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "model_22"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_15 (InputLayer)          [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1_pad (ZeroPadding2D)      (None, 257, 257, 3)  0           ['input_15[0][0]']               
                                                                                                  
 Conv1 (Conv2D)                 (None, 128, 128, 32  864         ['Conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

/usr/local/lib/python3.10/dist-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "model_23"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_16 (InputLayer)          [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1_pad (ZeroPadding2D)      (None, 257, 257, 3)  0           ['input_16[0][0]']               
                                                                                                  
 Conv1 (Conv2D)                 (None, 128, 128, 32  864         ['Conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

In [ ]:
from google.colab import runtime
runtime.unassign()

## Missing on resized folds 3 and 4 (see 3 cells above)